# Description
NordPool provides the spot prices for electricty in the nordic, however their historical data is locked behind a paywall. Luckily the danish government provides this data from NordPool for free. This means that we need to convert the spot price per MWh from EUR to SEK.

Upon validation against Vattenfalls data, which is not downloadable, there is a difference around 0.1 öre.
https://www.vattenfall.se/elavtal/elpriser/timpris/

Source of the spot data:
https://www.energidataservice.dk/tso-electricity/elspotprices#metadata-info

Source of the exchange rates:
https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-sek.en.html

In [5]:
import datetime


def parse_xml(path):
    data = {}
    with open(path, "r") as f:
        while True:
            line = f.readline().strip("\n")
            if not line:
                break

            if not line.startswith("<Obs"):
                continue

            _, time_period, obs_value, _, _, _ = line.split(" ")
            _, date_str = time_period.split("=")
            _, exchange_rate = obs_value.split("=")

            exchange_rate = exchange_rate.strip('"')
            date_str = date_str.strip('"')

            data[date_str] = float(exchange_rate)

    return data


def parse_csv(path, separator):
    data = {}
    with open(path, "r") as f:
        _ = f.readline()
        while True:
            line = f.readline().strip("\n")
            if not line:
                break
            _, datetime_dk, _, _, spot_price_eur = line.split(separator)
            date_dk, hour_dk = datetime_dk.split(" ")

            if date_dk not in data:
                data[date_dk] = {}

            if hour_dk not in data[date_dk]:
                data[date_dk][hour_dk] = {}
            data[date_dk][hour_dk] = float(spot_price_eur.replace(",", "."))

    return data


def fill_in_weekends(exchange_data):
    additional_data = {}
    for (date_str, rate) in exchange_data.items():
        date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
        tomorrow = date + datetime.timedelta(days=1)

        while True:
            if tomorrow > datetime.datetime(2024, 1, 2):
                break

            tomorrow_str = datetime.datetime.strftime(tomorrow, "%Y-%m-%d")
            if tomorrow_str in exchange_data:
                break

            additional_data[tomorrow_str] = rate
            tomorrow += datetime.timedelta(days=1)

    exchange_data.update(additional_data)
    return exchange_data


def format_csv(spot_prices, exchange_rate, start_date, stop_date, output_path):
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    stop_date = datetime.datetime.strptime(stop_date, "%Y-%m-%d")

    hours = [f"{i:02}:00" for i in range(0, 24)]
    header = f"date;{';'.join(hours)}"

    current_date = start_date
    with open(output_path, "w") as f:
        f.write(header + "\n")

        while current_date < stop_date:
            current_date_str = datetime.datetime.strftime(current_date, "%Y-%m-%d")
            rate = exchange_rate[current_date_str]

            spot_prices_sek = []
            for h in hours:
                spot_date = spot_prices[current_date_str]
                if h in spot_date:
                    spot_price_eur = spot_date[h]
                else:
                    spot_price_eur = 0.0
                spot_price_sek = spot_price_eur*rate
                spot_prices_sek.append(f"{spot_price_sek:.6f}")

            spot_prices_sek_str = ";".join(spot_prices_sek)
            line = f"{current_date_str};{spot_prices_sek_str}"
            f.write(line + "\n")

            current_date += datetime.timedelta(days=1)

In [7]:
elspotprices_path = "data/elspotprices.csv"
exchange_rate_path = "data/eur-2-sek-exchange.xml"
elspotprices = parse_csv(elspotprices_path, ";")
exchange_rate = parse_xml(exchange_rate_path)
exchange_rate = fill_in_weekends(exchange_rate)

output_file = "elspotprices_sek.csv"
format_csv(elspotprices, exchange_rate, "2016-01-01", "2024-01-01", output_file)

## The created csv files look

In [16]:
with open(output_file, "r") as f:
    for i in range(0, 30):
        line = f.readline().strip("\n")
        print(line[0:100] + "...")
print(".\n.\n.")

date;00:00;01:00;02:00;03:00;04:00;05:00;06:00;07:00;08:00;09:00;10:00;11:00;12:00;13:00;14:00;15:00...
2016-01-01;150.615896;147.399589;144.642730;143.080515;142.161565;144.734625;141.150720;141.242615;1...
2016-01-02;151.351056;146.021155;138.118185;121.209505;116.614755;118.912130;124.885305;130.950375;1...
2016-01-03;146.940105;145.837365;144.826520;143.264305;144.183255;145.469785;137.658710;142.712935;1...
2016-01-04;146.346816;144.971376;142.678976;142.770672;145.980032;153.682496;165.144496;179.173993;1...
2016-01-05;163.071480;159.843255;159.013140;159.751020;162.887010;165.285120;173.401791;189.173985;2...
2016-01-06;184.844660;181.243781;178.473890;178.658550;180.135830;182.905721;189.276500;203.679971;2...
2016-01-07;209.775589;201.796500;199.477000;199.662560;202.538740;224.527609;251.155460;670.428299;9...
2016-01-08;195.099831;186.762240;183.241929;183.983049;186.854880;192.042720;203.993280;277.086240;3...
2016-01-09;227.060640;217.148169;211.126569;208.347360;203.43743